In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, Imputer, MinMaxScaler, RobustScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from mlxtend.evaluate import lift_score
from sklearn.metrics import make_scorer

In [2]:
X = pd.read_csv('orange_small_churn_data_train.csv')

In [3]:
y = X.label
X.drop('label', axis=1, inplace=True)

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 49.1+ MB


In [5]:
cat_columns = X.select_dtypes(include='object').columns
X[cat_columns] = X[cat_columns].astype(np.str)
cat_columns

Index(['Var191', 'Var192', 'Var193', 'Var194', 'Var195', 'Var196', 'Var197',
       'Var198', 'Var199', 'Var200', 'Var201', 'Var202', 'Var203', 'Var204',
       'Var205', 'Var206', 'Var207', 'Var208', 'Var210', 'Var211', 'Var212',
       'Var213', 'Var214', 'Var215', 'Var216', 'Var217', 'Var218', 'Var219',
       'Var220', 'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229'],
      dtype='object')

In [6]:
# вспомогательная функция для кодирования всех категориальных признаков
class MultiColumnLabelEncoder:
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        res = X.copy()
        if self.columns is not None:
            for col in self.columns:
                res[col] = LabelEncoder().fit_transform(res[col])
        else:
            for colname, col in res.iteritems():
                res[colname] = LabelEncoder().fit_transform(col)
        return res

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [7]:
# категориальные признаки закодируем с помощью LabelEncoder
encoder = MultiColumnLabelEncoder(columns=cat_columns)
# пустые значения заменим на среднее ззначение признака
imputer = Imputer(missing_values=np.NAN, axis=1)
# отмасштабируем признаки
scaler = MinMaxScaler()
# исбавимся от выбросов после масщтабирования, если они будет в тестовой выборке
outlier = RobustScaler()
# для уменьшения несбаллансированности классов воспользуемся техникой Oversampling
smote = SMOTE(random_state=17)

In [8]:
# будем использовать стратифицирование разбиение на 10 фолдов
cv = StratifiedKFold(n_splits=10, random_state=17)

In [9]:
# на данном этапе тонкую настройку параметров производить не будем
models = [
    ('Ridge', RidgeClassifier(random_state=17)),
    ('RandomForest', RandomForestClassifier(n_estimators=300, random_state=17)),
    ('XGBoost', XGBClassifier(n_estimators=300, random_state=17))
]

In [10]:
# наши основные и вспомогательгные метрики из задания 2-й недели
scores = {
    'PR-AUC': 'average_precision',
    'ROC-AUC': 'roc_auc',
    'F1': 'f1',
    'Precision': 'precision',
    'Recall': 'recall',
    'Lift': make_scorer(lift_score)
}

In [11]:
for model_name, model in models:
    print(model_name)
    clf = Pipeline([
        ('encoder' , encoder),
        ('imputer' , imputer),
        ('scaler'  , scaler),
        ('outlier', outlier),
        ('smote', smote),
        (model_name, model)
    ])
    cv_res = cross_validate(clf, X, y, cv=cv.split(X, y), scoring=scores, n_jobs=1, return_train_score=False)
    for score in scores:
        test_scores = cv_res['test_'+score]
        print('%s: %.3f ± %.3f' % (score, np.mean(test_scores), np.std(test_scores)), end='\t')
    print()

Ridge
PR-AUC: 0.140 ± 0.014	ROC-AUC: 0.654 ± 0.019	F1: 0.180 ± 0.020	Precision: 0.105 ± 0.016	Recall: 0.700 ± 0.161	Lift: 1.409 ± 0.215	
RandomForest
PR-AUC: 0.134 ± 0.008	ROC-AUC: 0.670 ± 0.014	F1: 0.064 ± 0.045	Precision: 0.172 ± 0.087	Recall: 0.043 ± 0.036	Lift: 2.312 ± 1.172	
XGBoost
PR-AUC: 0.173 ± 0.017	ROC-AUC: 0.703 ± 0.013	F1: 0.135 ± 0.042	Precision: 0.291 ± 0.060	Recall: 0.096 ± 0.048	Lift: 3.911 ± 0.812	


Из baseline моделей лучший результат показал градиентный бустинг.
Качество градиентного бустинга и случайного леса скорее всего получится улучшить за счет тонкой настройки параметров.
Качество линейной модели после тонкой настройки параметров скорее всего существенно не изменится.